# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [40]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?currentJobId=4187409133&'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    results = soup.select('ul.jobs-search__results-list')[0] # get list with cards
    cards = results.select("div.base-search-card__info") # get each card individualls
    for card in cards:
        title = card.find("h3", class_="base-search-card__title").get_text(strip=True)
        company = card.find("h4", class_="base-search-card__subtitle").get_text(strip=True)
        location = card.find("span", class_="job-search-card__location").get_text(strip=True)
        titles.append(title)
        companies.append(company)
        locations.append(location)
    data = pd.DataFrame({
        "title":titles,
        "company":companies,
        "location":locations
    })
    
    # Return dataframe
    return data

In [41]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,title,company,location
0,"Analyst, Data Analytics",Bath & Body Works,"Columbus, OH"
1,Quant Analyst Internship,OptionMetrics,"New York, NY"
2,"Senior Manager, Analytics and Business Intelli...",RevolutionParts,United States
3,Analyst - Demand Generation,Freshworks,"San Mateo, CA"
4,CRM Assistant,Harry Winston,"New York, United States"
5,Director - Business Analytics,Freshworks,"San Mateo, CA"
6,Quantitative Analyst,bp,"New York, NY"
7,"Data Scientist, GTM",Notion,"San Francisco, CA"
8,Data Analysis Specialist,SkyWest Airlines,"Salt Lake City, UT"
9,Data Analyst I,BlueCross BlueShield of South Carolina,"Columbia, SC"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [44]:
"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search_pages(keywords, pages):
    
    listing_count = 0
    titles = []
    companies = []
    locations = []
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?currentJobId=4187409133&'

    for _ in range(pages):
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', str(listing_count)])
        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        results = soup.select('ul.jobs-search__results-list')[0] # get list with cards
        cards = results.select("div.base-search-card__info") # get each card individualls
        for card in cards:
            title = card.find("h3", class_="base-search-card__title").get_text(strip=True)
            company = card.find("h4", class_="base-search-card__subtitle").get_text(strip=True)
            location = card.find("span", class_="job-search-card__location").get_text(strip=True)
            titles.append(title)
            companies.append(company)
            locations.append(location)
            listing_count += 1
    data = pd.DataFrame({
        "title":titles,
        "company":companies,
        "location":locations
    })
    
    # Return dataframe
    return data

In [45]:
# Example to call the function

results = scrape_linkedin_job_search_pages('data%20analysis', 5)
results

,title,company,location
0,"Analyst, Data Analytics",Bath & Body Works,"Columbus, OH"
1,Quant Analyst Internship,OptionMetrics,"New York, NY"
2,"Senior Manager, Analytics and Business Intelli...",RevolutionParts,United States
3,Analyst - Demand Generation,Freshworks,"San Mateo, CA"
4,Director - Business Analytics,Freshworks,"San Mateo, CA"
...,...,...,...
295,Data Analyst (Alteryx and SQL),Acumenz Consulting,United States
296,Data Analyst Intern,InComm Payments,"Atlanta, GA"
297,"Data Analyst II ( SQL, Excel, Python)",Sunshine Health,"Florida, United States"
298,Data Analyst,Strategic Staffing Solutions,"Tampa, FL"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [46]:
"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search_pages_country(keywords, pages, country):
    
    listing_count = 0
    titles = []
    companies = []
    locations = []
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?currentJobId=4187409133&'

    for _ in range(pages):
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', str(listing_count), '&location=', country])
        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        results = soup.select('ul.jobs-search__results-list')[0] # get list with cards
        cards = results.select("div.base-search-card__info") # get each card individualls
        for card in cards:
            title = card.find("h3", class_="base-search-card__title").get_text(strip=True)
            company = card.find("h4", class_="base-search-card__subtitle").get_text(strip=True)
            location = card.find("span", class_="job-search-card__location").get_text(strip=True)
            titles.append(title)
            companies.append(company)
            locations.append(location)
            listing_count += 1
    data = pd.DataFrame({
        "title":titles,
        "company":companies,
        "location":locations
    })
    
    # Return dataframe
    return data

In [47]:
# Example to call the function

results = scrape_linkedin_job_search_pages_country('data%20analysis', 5, "Germany")
results

,title,company,location
0,Analytics & Insights Manager – Graduate Entry ...,Procter & Gamble,"Schwalbach, Hesse, Germany"
1,Consumer Insights Senior Analyst,SharkNinja,"Frankfurt, Hesse, Germany"
2,Private Equity Analyst,Flynn and Chase,"Hamburg, Germany"
3,(Junior) Analytics Engineer,Gemma Analytics,"Berlin, Berlin, Germany"
4,Investment Banking Analyst,PeakSeek,"Munich, Bavaria, Germany"
...,...,...,...
295,(Junior) Business Analyst (m/w/d) Controlling,Nexis,"Regensburg, Bavaria, Germany"
296,Senior Credit Risk Modeller (m/w/d),BAUMLINK,Germany
297,Investment Analyst | Single Family Office,THRONSBERG | Private Capital Recruitment,"Hamburg, Germany"
298,"Pricing Analyst (m,w,d) Remote /Flex",Insulet Corporation,"Munich, Bavaria, Germany"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [62]:
"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search_pages_country_time(keywords, pages, country, days):
    
    listing_count = 0
    titles = []
    companies = []
    locations = []
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?currentJobId=4187409133&'

    for _ in range(pages):
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', str(listing_count), '&location=', country,
                              '&f_TPR=r', str(days*24*60*60)])
        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        results = soup.select('ul.jobs-search__results-list')[0] # get list with cards
        cards = results.select("div.base-search-card__info") # get each card individualls
        for card in cards:
            title = card.find("h3", class_="base-search-card__title").get_text(strip=True)
            company = card.find("h4", class_="base-search-card__subtitle").get_text(strip=True)
            location = card.find("span", class_="job-search-card__location").get_text(strip=True)
            titles.append(title)
            companies.append(company)
            locations.append(location)
            listing_count += 1
    data = pd.DataFrame({
        "title":titles,
        "company":companies,
        "location":locations
    })
    
    # Return dataframe
    return data

In [63]:
# Example to call the function

results = scrape_linkedin_job_search_pages_country_time('data%20analysis', 2, "Germany", 7)
results

,title,company,location
0,"Investment Banking, Mergers and Acquisitions A...",Jefferies,"Frankfurt, Hesse, Germany"
1,Data Analyst (B2C) (m/f/d),Omio,"Berlin, Berlin, Germany"
2,(Senior) Strategy Associate (Management Consul...,Crypto.com,"Berlin, Berlin, Germany"
3,Graduate – Data Management – based in Luxembourg,European Investment Bank (EIB),"Berlin, Berlin, Germany"
4,Consumer Insights Senior Analyst,SharkNinja,"Frankfurt, Hesse, Germany"
...,...,...,...
115,Junior Research Analyst,Cushman & Wakefield,"Berlin, Berlin, Germany"
116,Credit Risk Analyst,Vallum Associates,"Essen, North Rhine-Westphalia, Germany"
117,M&A Analyst (m/w/d)*,Comanos,"North Rhine-Westphalia, Germany"
118,(Junior) Business Analyst (m/w/d) Controlling,Nexis,"Regensburg, Bavaria, Germany"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [94]:
"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search_pages_country_time_seniority(keywords, pages, country, days):
    
    listing_count = 0
    titles = []
    companies = []
    locations = []
    links = []
    seniority = []
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?currentJobId=4187409133&'

    for _ in range(pages):
        # Assemble the full url with parameters
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', str(listing_count), '&location=', country,
                              '&f_TPR=r', str(days*24*60*60)])
        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

        results = soup.select('ul.jobs-search__results-list')[0] # get list with cards
        cards = results.select("div.base-card") # get each card individualls
        for card in cards:
            title = card.find("h3", class_="base-search-card__title").get_text(strip=True)
            company = card.find("h4", class_="base-search-card__subtitle").get_text(strip=True)
            location = card.find("span", class_="job-search-card__location").get_text(strip=True)

            link = card.find("a").get("href")
            link_access = requests.get(link)
            soup_link = BeautifulSoup(link_access.text, 'html.parser')
            seniority_level = soup_link.find('span', class_=[
                                        'description__job-criteria-text',
                                        'description__job-criteria-text--criteria'
                                    ])


            titles.append(title)
            companies.append(company)
            locations.append(location)
            links.append(link)
            seniority.append(seniority_level.get_text(strip=True))
            listing_count += 1
    data = pd.DataFrame({
        "title":titles,
        "company":companies,
        "location":locations,
        "seniority": seniority,
        "page":links
    })
    
    # Return dataframe
    return data

In [95]:
# Example to call the function

results = scrape_linkedin_job_search_pages_country_time_seniority('data%20analysis', 1, "Germany", 7)
results

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))